In [1]:
from efx import Allocation, generate_valuations
import itertools
import numpy as np

In [12]:
def mms_value(valuations, agent):
    n = len(valuations)
    items = list(range(len(valuations[0])))
    max_min = 0
    for partition in itertools.product(range(n), repeat=len(items)):
        bundles = [[] for _ in range(n)]
        for item, idx in zip(items, partition):
            bundles[idx].append(item)
        # For each partition, compute the worst bundle the agent could get
        min_val = min(sum(valuations[agent][i] for i in bundle) for bundle in bundles)
        max_min = max(max_min, min_val)
    return max_min

def compute_mms(valuations, agent):
    return mms_value(valuations, agent)

In [3]:
def satisfies_mms(allocation, factor=2/3):
    for i in range(allocation.n):
        mms = mms_value(allocation.valuations, i)
        if allocation.value(i) < factor * mms:
            return False
    return True

In [8]:
def threshold_graph(valuations, bundles, thresholds):
    """Returns bipartite edges (i, j) where agent i values bundle j ≥ threshold."""
    n = len(valuations)
    edges = []
    for i in range(n):
        for j in range(n):
            if sum(valuations[i][item] for item in bundles[j]) >= thresholds[i]:
                edges.append((i, j))
    return edges

In [9]:
def find_matching(n, edges):
    """Greedy matching: returns match dict i → j such that each i and j is matched once."""
    from collections import defaultdict

    # Build adjacency
    adj = defaultdict(list)
    for i, j in edges:
        adj[i].append(j)

    matched_i = set()
    matched_j = set()
    match = {}
    
    for i in range(n):
        for j in adj[i]:
            if j not in matched_j:
                match[i] = j
                matched_i.add(i)
                matched_j.add(j)
                break
    return match

In [17]:
def is_efx_with_charity(allocation):
    """Returns True if some subset of agents has an EFX allocation (e.g., after removing empty-bundle agents)."""
    non_empty_agents = [i for i in range(allocation.n) if allocation.bundles[i]]
    
    if len(non_empty_agents) <= 1:
        return True  # Vacuously true
    
    sub_bundles = [allocation.bundles[i] for i in non_empty_agents]
    sub_valuations = [allocation.valuations[i] for i in non_empty_agents]
    
    sub_allocation = Allocation(sub_bundles, sub_valuations)
    return sub_allocation.is_efx()

In [10]:
def approxMMS(valuations):
    """
    Implements Algorithm 1: approxMMS
    valuations: list of list of integers (n x m), where n agents and m items
    returns: bundles (allocation), a list of n lists
    """
    n = len(valuations)
    items = list(range(len(valuations[0])))
    remaining_agents = list(range(n))
    remaining_items = items.copy()
    allocation = [[] for _ in range(n)]
    mms_values = [compute_mms(valuations, i) for i in range(n)]

    while remaining_agents:
        n_rem = len(remaining_agents)

        # Step 5: Partition items into n bundles
        # We brute-force for now: try all partitions and pick one satisfying the threshold
        best_partition = None
        for partition in itertools.product(range(n_rem), repeat=len(remaining_items)):
            bundles = [[] for _ in range(n_rem)]
            for item, a in zip(remaining_items, partition):
                bundles[a].append(item)
            valid = True
            for i in range(n_rem):
                if sum(valuations[remaining_agents[i]][item] for item in bundles[i]) < (2/3) * mms_values[remaining_agents[i]]:
                    valid = False
                    break
            if valid:
                best_partition = bundles
                break

        if best_partition is None:
            raise ValueError("No valid 2/3 MMS partition found.")

        # Step 6: Build threshold graph
        t_vals = [(2/3) * mms_values[i] for i in remaining_agents]
        edges = threshold_graph(valuations, best_partition, t_vals)

        # Step 7: Find matching
        match = find_matching(n_rem, edges)
        if not match:
            raise ValueError("No matching found in threshold graph.")

        matched_agents = list(match.keys())
        matched_bundles = [match[i] for i in matched_agents]

        for i in matched_agents:
            real_agent = remaining_agents[i]
            real_bundle = best_partition[match[i]]
            allocation[real_agent] = real_bundle

        # Step 8: Update agent and item pools
        unmatched = [i for i in range(n_rem) if i not in matched_agents]
        remaining_agents = [remaining_agents[i] for i in unmatched]
        allocated = set()
        for j in range(n_rem):
            if j not in unmatched:
                allocated.update(best_partition[j])
        remaining_items = [item for item in remaining_items if item not in allocated]

    return allocation

In [ ]:
from itertools import product
def test_approxMMS():
    n_agents = 3
    n_items = 6
    valuations = generate_valuations(n_agents, n_items, seed=42)

    print("Valuations:")
    for i, row in enumerate(valuations):
        print(f"Agent {i}: {row}")

    try:
        bundles = approxMMS(valuations)
    except Exception as e:
        print(f"Algorithm failed: {e}")
        return

    allocation = Allocation(bundles, valuations)

    print("\nAllocation:")
    for i, bundle in enumerate(allocation.bundles):
        print(f"  Agent {i}: Items {bundle} | Value: {allocation.value(i)}")

    print("EFX Check:", "Yes" if allocation.is_efx() else "No")

    efx_charity = is_efx_with_charity(allocation)
    print("EFX with Charity Check:", "Yes" if efx_charity else "No")

    # MMS validation
    def compute_mms(valuations, agent):
        n = len(valuations)
        items = list(range(len(valuations[0])))
        max_min = 0
        for partition in product(range(n), repeat=len(items)):
            bundles = [[] for _ in range(n)]
            for item, idx in zip(items, partition):
                bundles[idx].append(item)
            min_val = min(sum(valuations[agent][i] for i in bundle) for bundle in bundles)
            max_min = max(max_min, min_val)
        return max_min

    mms_satisfied = True
    for i in range(n_agents):
        mms = compute_mms(valuations, i)
        agent_val = allocation.value(i)
        print(f"Agent {i}: Value = {agent_val}, MMS = {mms}, 2/3 MMS = {2/3 * mms}")
        if agent_val < (2/3) * mms:
            mms_satisfied = False

    print("2/3-MMS Check:", "Yes" if mms_satisfied else "No")

if __name__ == "__main__":
    test_approxMMS()

Valuations:
Agent 0: [7, 4, 8, 5, 7, 10]
Agent 1: [3, 7, 8, 5, 4, 8]
Agent 2: [8, 3, 6, 5, 2, 8]

Allocation:
  Agent 0: Items [0, 1, 2] | Value: 19
  Agent 1: Items [3, 4] | Value: 9
  Agent 2: Items [5] | Value: 8
EFX Check: ❌ No
EFX with Charity Check: ❌ No
Agent 0: Value = 19, MMS = 13, 2/3 MMS = 8.666666666666666
Agent 1: Value = 9, MMS = 11, 2/3 MMS = 7.333333333333333
Agent 2: Value = 8, MMS = 10, 2/3 MMS = 6.666666666666666
2/3-MMS Check: ✅ Yes


In [7]:
n_agents, n_items = 3, 5
valuations = generate_valuations(n_agents, n_items, seed=42)
print('Valuations:', valuations)

items = list(range(n_items))
efx_mms_allocations = []

for assignment in itertools.product(range(n_agents), repeat=n_items):
    bundles = [[] for _ in range(n_agents)]
    for item, agent in zip(items, assignment):
        bundles[agent].append(item)
    alloc = Allocation(bundles, valuations)
    if alloc.is_efx() and satisfies_mms(alloc):
        efx_mms_allocations.append(alloc)

print(f"The MMS value is computed as the maximum of the minimum values across all possible partitions for each agent.")
print(f"The MMS value is " + str(mms_value(valuations, 1)) + " for agent 1.")
print(f"The MMS value is " + str(mms_value(valuations, 2)) + " for agent 2.")
print(f"The MMS value is " + str(mms_value(valuations, 0)) + " for agent 0.")
print(f"\nFound {len(efx_mms_allocations)} allocations that are EFX and 2/3-MMS.\n")
for i, alloc in enumerate(efx_mms_allocations[:3]):
    print(f"Allocation {i+1}:")
    for j in range(alloc.n):
        print(f"  Agent {j}: {alloc.bundles[j]} (value = {alloc.value(j)})")

Valuations: [[7, 4, 8, 5, 7], [10, 3, 7, 8, 5], [4, 8, 8, 3, 6]]
The MMS value is computed as the maximum of the minimum values across all possible partitions for each agent.
The MMS value is 10 for agent 1.
The MMS value is 8 for agent 2.
The MMS value is 8 for agent 0.

Found 22 allocations that are EFX and 2/3-MMS.

Allocation 1:
  Agent 0: [0, 1] (value = 11)
  Agent 1: [3, 4] (value = 13)
  Agent 2: [2] (value = 8)
Allocation 2:
  Agent 0: [0, 4] (value = 14)
  Agent 1: [1, 3] (value = 11)
  Agent 2: [2] (value = 8)
Allocation 3:
  Agent 0: [0, 2] (value = 15)
  Agent 1: [3, 4] (value = 13)
  Agent 2: [1] (value = 8)
